# Module 9: Classification Metrics

**Goal:** Understand precision, recall, and why accuracy can be misleading. Learn to choose the right metric for your business problem.

**Prerequisites:** Module 4 (Logistic Regression)

**Expected Runtime:** ~25 minutes

**Outputs:**
- Confusion matrix analysis
- ROC and PR curves
- Threshold optimization

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score,
    precision_recall_curve, average_precision_score
)
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## Part 1: Generate Imbalanced Churn Data

Real-world classification problems are often imbalanced. Let's create a dataset where only 10% of customers churn.

In [ ]:
n_samples = 2000
churn_rate = 0.10  # 10% churn

# Features
tenure = np.random.uniform(0, 60, n_samples)
monthly_charges = np.random.uniform(20, 120, n_samples)
support_tickets = np.random.poisson(2, n_samples)
usage_decline = np.random.uniform(-20, 50, n_samples)

# Churn probability (logistic relationship)
logit = -3 + (0.02 * usage_decline) + (0.1 * support_tickets) - (0.03 * tenure) + (0.01 * monthly_charges)
prob = 1 / (1 + np.exp(-logit))
churn = (np.random.random(n_samples) < prob).astype(int)

df = pd.DataFrame({
    'tenure': tenure,
    'monthly_charges': monthly_charges,
    'support_tickets': support_tickets,
    'usage_decline': usage_decline,
    'churn': churn
})

print(f"Dataset shape: {df.shape}")
print(f"\nChurn distribution:")
print(df['churn'].value_counts(normalize=True).round(3))

## Part 2: Train a Model and Get Probabilities

In [ ]:
X = df[['tenure', 'monthly_charges', 'support_tickets', 'usage_decline']]
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Get probabilities (not just predictions)
y_prob = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)  # Default threshold = 0.5

print(f"Test set: {len(y_test)} samples")
print(f"Actual churners: {y_test.sum()} ({y_test.mean()*100:.1f}%)")
print(f"Predicted churners (threshold=0.5): {y_pred.sum()}")

## Part 3: The Accuracy Trap

Let's see why accuracy can be misleading.

In [ ]:
# Calculate accuracy
accuracy = (y_pred == y_test).mean()

# What if we predicted NO CHURN for everyone?
naive_pred = np.zeros_like(y_test)
naive_accuracy = (naive_pred == y_test).mean()

print("=== The Accuracy Trap ===")
print(f"Our model accuracy: {accuracy*100:.1f}%")
print(f"Naive 'no churn' accuracy: {naive_accuracy*100:.1f}%")
print(f"\n⚠️  Predicting 'no churn' for everyone gets {naive_accuracy*100:.1f}% accuracy!")
print(f"    But catches 0% of actual churners.")

## Part 4: Confusion Matrix Deep Dive

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

print("=== Confusion Matrix ===")
print(f"\n              Predicted")
print(f"              No Churn    Churn")
print(f"Actual No    TN={tn:<6}  FP={fp}")
print(f"Actual Yes   FN={fn:<6}  TP={tp}")

print("\n=== Interpretation ===")
print(f"True Positives (TP):  {tp} - Correctly identified churners")
print(f"True Negatives (TN):  {tn} - Correctly identified loyal customers")
print(f"False Positives (FP): {fp} - Wrongly flagged as churners (wasted outreach)")
print(f"False Negatives (FN): {fn} - Missed churners (lost customers!)")

# Visualize
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, cmap='Blues')

ax.set_xticks([0, 1])
ax.set_yticks([0, 1])
ax.set_xticklabels(['Predicted: No Churn', 'Predicted: Churn'])
ax.set_yticklabels(['Actual: No Churn', 'Actual: Churn'])

labels = [['TN', 'FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        text = f"{labels[i][j]}\n{cm[i,j]}"
        ax.text(j, i, text, ha='center', va='center', fontsize=14, 
                color='white' if cm[i,j] > cm.max()/2 else 'black')

ax.set_title('Confusion Matrix')
plt.tight_layout()
plt.show()

## Part 5: Precision, Recall, and F1

In [ ]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("=== Key Metrics (at threshold=0.5) ===")
print(f"\nPrecision: {precision:.3f}")
print(f"  → Of {y_pred.sum()} predicted churners, {tp} were actual churners")
print(f"  → 'When we flag someone, we're right {precision*100:.0f}% of the time'")

print(f"\nRecall: {recall:.3f}")
print(f"  → Of {y_test.sum()} actual churners, we caught {tp}")
print(f"  → 'We catch {recall*100:.0f}% of all churners'")

print(f"\nF1 Score: {f1:.3f}")
print(f"  → Harmonic mean of precision and recall")

print("\n" + "="*50)
print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

## Part 6: The Threshold Effect

The default threshold of 0.5 is rarely optimal. Let's explore.

In [ ]:
# Calculate metrics at different thresholds
thresholds = np.arange(0.1, 0.9, 0.05)
results = []

for t in thresholds:
    y_pred_t = (y_prob >= t).astype(int)
    cm_t = confusion_matrix(y_test, y_pred_t)
    tn_t, fp_t, fn_t, tp_t = cm_t.ravel()
    
    prec = tp_t / (tp_t + fp_t) if (tp_t + fp_t) > 0 else 0
    rec = tp_t / (tp_t + fn_t) if (tp_t + fn_t) > 0 else 0
    f1_t = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0
    
    results.append({
        'threshold': t,
        'precision': prec,
        'recall': rec,
        'f1': f1_t,
        'predicted_positive': y_pred_t.sum()
    })

results_df = pd.DataFrame(results)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(results_df['threshold'], results_df['precision'], 'b-', linewidth=2, label='Precision')
ax1.plot(results_df['threshold'], results_df['recall'], 'g-', linewidth=2, label='Recall')
ax1.plot(results_df['threshold'], results_df['f1'], 'r--', linewidth=2, label='F1')
ax1.axvline(x=0.5, color='gray', linestyle=':', label='Default (0.5)')
ax1.set_xlabel('Threshold')
ax1.set_ylabel('Score')
ax1.set_title('Precision-Recall Tradeoff')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.bar(results_df['threshold'], results_df['predicted_positive'], width=0.04, color='steelblue')
ax2.axhline(y=y_test.sum(), color='red', linestyle='--', label=f'Actual churners ({y_test.sum()})')
ax2.set_xlabel('Threshold')
ax2.set_ylabel('Number Predicted Positive')
ax2.set_title('How Many We Flag')
ax2.legend()

plt.tight_layout()
plt.show()

print("\n📊 Key Insight: Lower threshold = more flags = higher recall but lower precision")

## Part 7: ROC Curve and AUC

In [ ]:
# Calculate ROC curve
fpr, tpr, roc_thresholds = roc_curve(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob)

# Calculate PR curve
precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_test, y_prob)
pr_auc = average_precision_score(y_test, y_prob)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC Curve
ax1 = axes[0]
ax1.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {roc_auc:.3f})')
ax1.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.5)')
ax1.fill_between(fpr, tpr, alpha=0.2)
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate (Recall)')
ax1.set_title('ROC Curve')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# PR Curve
ax2 = axes[1]
ax2.plot(recall_curve, precision_curve, 'g-', linewidth=2, label=f'PR (AP = {pr_auc:.3f})')
ax2.axhline(y=y_test.mean(), color='k', linestyle='--', label=f'Random ({y_test.mean():.2f})')
ax2.fill_between(recall_curve, precision_curve, alpha=0.2, color='green')
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision-Recall Curve')
ax2.legend(loc='lower left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n=== Threshold-Independent Metrics ===")
print(f"ROC-AUC: {roc_auc:.3f} - Model ranks churners higher than non-churners {roc_auc*100:.0f}% of the time")
print(f"PR-AUC:  {pr_auc:.3f} - Better metric for imbalanced data")

## Part 8: TODO - Cost-Based Threshold Optimization

If you know the business costs of errors, you can find the optimal threshold.

In [ ]:
# TODO: Set your business costs
# Cost of missing a churner (FN): lost customer revenue
cost_fn = 500  # $ lost per missed churner

# Cost of false alarm (FP): wasted marketing
cost_fp = 30   # $ wasted per unnecessary outreach

# Calculate total cost at each threshold
threshold_costs = []

for t in np.arange(0.05, 0.95, 0.01):
    y_pred_t = (y_prob >= t).astype(int)
    cm_t = confusion_matrix(y_test, y_pred_t)
    tn_t, fp_t, fn_t, tp_t = cm_t.ravel()
    
    total_cost = (fn_t * cost_fn) + (fp_t * cost_fp)
    
    threshold_costs.append({
        'threshold': t,
        'total_cost': total_cost,
        'fn_cost': fn_t * cost_fn,
        'fp_cost': fp_t * cost_fp,
        'fn': fn_t,
        'fp': fp_t
    })

cost_df = pd.DataFrame(threshold_costs)
optimal_idx = cost_df['total_cost'].idxmin()
optimal_threshold = cost_df.loc[optimal_idx, 'threshold']

# Plot
plt.figure(figsize=(10, 6))
plt.plot(cost_df['threshold'], cost_df['total_cost'], 'b-', linewidth=2, label='Total Cost')
plt.plot(cost_df['threshold'], cost_df['fn_cost'], 'r--', linewidth=1, label='FN Cost (missed churners)')
plt.plot(cost_df['threshold'], cost_df['fp_cost'], 'g--', linewidth=1, label='FP Cost (wasted outreach)')
plt.axvline(x=optimal_threshold, color='orange', linestyle='-', linewidth=2, label=f'Optimal ({optimal_threshold:.2f})')
plt.axvline(x=0.5, color='gray', linestyle=':', label='Default (0.5)')
plt.xlabel('Threshold')
plt.ylabel('Cost ($)')
plt.title('Cost-Based Threshold Optimization')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n=== Cost-Based Analysis ===")
print(f"Cost assumptions: FN=${cost_fn}, FP=${cost_fp}")
print(f"Optimal threshold: {optimal_threshold:.2f}")
print(f"Expected theoretical optimal: {cost_fp / (cost_fp + cost_fn):.2f}")

# Compare default vs optimal
default_cost = cost_df[cost_df['threshold'] == 0.5]['total_cost'].values[0] if 0.5 in cost_df['threshold'].values else cost_df.loc[(cost_df['threshold'] - 0.5).abs().idxmin(), 'total_cost']
optimal_cost = cost_df.loc[optimal_idx, 'total_cost']

print(f"\nCost at default (0.5): ${default_cost:,.0f}")
print(f"Cost at optimal ({optimal_threshold:.2f}): ${optimal_cost:,.0f}")
print(f"Savings: ${default_cost - optimal_cost:,.0f}")

## Part 9: Stakeholder Summary

### TODO: Write a 3-bullet summary (~100 words) for the marketing team

Template:
• **Model quality:** ROC-AUC = ____, meaning the model ranks churners above non-churners ____% of the time.
• **Recommended threshold:** Use ____ instead of 0.5 because [cost reason]. This catches ____% of churners (recall).
• **Expected impact:** At this threshold, expected cost savings are $____ compared to [baseline].

### Your Summary:

*Write your stakeholder summary here...*

---

## Self-Check

Uncomment and run the asserts below to verify your classification metrics are correct.

In [ ]:
# SELF-CHECK: Verify your classification metrics
assert 0.5 <= roc_auc <= 1.0, f"ROC-AUC should be between 0.5 and 1.0, got {roc_auc:.3f}"
assert 0 < pr_auc <= 1.0, f"PR-AUC should be between 0 and 1.0, got {pr_auc:.3f}"
assert cm.shape == (2, 2), "Confusion matrix should be 2x2"
assert 0.05 <= optimal_threshold <= 0.95, f"Optimal threshold should be reasonable, got {optimal_threshold:.2f}"
print(f"✅ Self-check passed! ROC-AUC: {roc_auc:.3f}, PR-AUC: {pr_auc:.3f}, Threshold: {optimal_threshold:.2f}")

## Key Takeaways

1. **Accuracy is often meaningless** on imbalanced data
2. **Precision** = "When I flag, am I right?" (use when FP is costly)
3. **Recall** = "Did I catch all positives?" (use when FN is costly)
4. **Threshold** is a business decision, not a model decision
5. **ROC-AUC** measures ranking ability; **PR-AUC** is better for imbalanced data

### Next Steps
- Explore the interactive playground to see the precision-recall tradeoff
- Complete the quiz to test your understanding